In [1]:
import pandas as pd
import os
import numpy as np

def read_csv_files_to_columns(folder_path):
    all_files = os.listdir(folder_path)
    csv_files = [f for f in all_files if f.endswith('.csv')]
    
    combined_df = pd.DataFrame()
    
    for i, file in enumerate(csv_files, start=1):
        file_path = os.path.join(folder_path, file)
        # Read CSV file with the 'High' header
        df_temp = pd.read_csv(file_path, usecols=['High'])
        # Rename the column to reflect its file origin
        df_temp.rename(columns={'High': f'High_{i}'}, inplace=True)
        combined_df = pd.concat([combined_df, df_temp], axis=1)
    
    return combined_df

pumpFolder = "C:/Users/mahrouaa/CS283_Project/DataPump/CS283/new_1m_binance"
randomFolder = "C:/Users/mahrouaa/CS283_Project/DataPump/CS283/random1m"

pump_df = read_csv_files_to_columns(pumpFolder)
random_df = read_csv_files_to_columns(randomFolder)
pump_df = pump_df.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
random_df = random_df.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

pump_df = pump_df.iloc[:, :466]
random_df = random_df.iloc[:, :466]

pump_series = pump_df.to_numpy().T
random_series = random_df.to_numpy().T


In [2]:
pump_df

,High_1,High_2,High_3,High_4,High_5,High_6,High_7,High_8,High_9,High_10,...,High_457,High_458,High_459,High_460,High_461,High_462,High_463,High_464,High_465,High_466
0,0.128713,0.240000,0.04,0.111111,0.174242,0.513514,0.069767,0.235294,0.071090,0.006623,...,0.017241,0.093079,0.015385,0.222672,0.011811,0.003195,0.117347,0.000000,0.014989,0.108696
1,0.128713,0.243333,0.04,0.111111,0.200758,0.513514,0.093023,0.235294,0.071090,0.006623,...,0.025862,0.093079,0.015385,0.287449,0.025591,0.003195,0.117347,0.000000,0.014989,0.108696
2,0.178218,0.240000,0.04,0.111111,0.200758,0.513514,0.069767,0.235294,0.071090,0.006623,...,0.025862,0.093079,0.015385,0.287449,0.027559,0.003195,0.117347,0.000000,0.014989,0.108696
3,0.158416,0.240000,0.04,0.111111,0.200758,0.513514,0.069767,0.235294,0.071090,0.006623,...,0.034483,0.064439,0.015385,0.287449,0.027559,0.003195,0.117347,0.004098,0.014989,0.112319
4,0.158416,0.240000,0.04,0.111111,0.200758,0.513514,0.069767,0.235294,0.071090,0.006623,...,0.043103,0.064439,0.015385,0.287449,0.027559,0.003195,0.112245,0.004098,0.014989,0.112319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.366337,0.013333,0.10,0.500000,0.348485,0.000000,0.139535,0.176471,0.180095,0.426049,...,0.620690,0.085919,0.220000,0.101215,0.411417,0.217252,0.173469,0.250000,0.289079,0.010870
996,0.366337,0.013333,0.12,0.500000,0.337121,0.000000,0.116279,0.176471,0.175355,0.443709,...,0.612069,0.085919,0.218462,0.101215,0.409449,0.207668,0.178571,0.245902,0.291221,0.000000
997,0.366337,0.013333,0.12,0.500000,0.344697,0.000000,0.116279,0.176471,0.175355,0.456954,...,0.620690,0.085919,0.218462,0.101215,0.407480,0.198083,0.178571,0.245902,0.291221,0.000000
998,0.366337,0.016667,0.12,0.500000,0.337121,0.000000,0.116279,0.176471,0.175355,0.465784,...,0.620690,0.085919,0.218462,0.101215,0.405512,0.217252,0.173469,0.245902,0.289079,0.007246


In [3]:
type(pump_series)

numpy.ndarray

# Pump and non-pump

In [4]:
import numpy as np
import pandas as pd
import os
from scipy.spatial.distance import cityblock, cosine
from numpy.linalg import norm
from scipy.stats import pearsonr, wasserstein_distance, ks_2samp, moment, skew, kurtosis
from tqdm import tqdm

def calculate_metrics(series1, series2):
    euclidean_distance = norm(series1 - series2)
    manhattan_distance = cityblock(series1, series2)
    cosine_sim = 1 - cosine(series1, series2)
    correlation, _ = pearsonr(series1, series2)
    emd = wasserstein_distance(series1, series2)
    ks_stat, ks_pvalue = ks_2samp(series1, series2)
    return euclidean_distance, manhattan_distance, cosine_sim, correlation, emd, ks_stat, ks_pvalue

def calculate_group_metrics(group1, group2):
    distances = []
    for series1 in tqdm(group1):
        for series2 in group2:
            metrics = calculate_metrics(series1, series2)
            distances.append(metrics)
    distances = np.array(distances)
    means = np.mean(distances, axis=0)
    stds = np.std(distances, axis=0)
    return means, stds

def print_metrics(title, means, stds):
    metrics_names = ["Euclidean", "Manhattan", "Cosine Similarity", "Pearson Correlation", "Earth Mover's", "KS Statistic"]
    print(title + " Averages and Standard Deviations:")
    for i, name in enumerate(metrics_names):
        print(f"{name}:")
        print("  Mean:", means[i], "  Standard Deviation:", stds[i])

print_metrics("Cross-Group", means_cross, stds_cross)


NameError: name 'means_cross' is not defined

In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis

def calculate_moments(series):
    return {
        'mean': np.mean(series),
        'variance': np.var(series),
        'skewness': skew(series),
        'kurtosis': kurtosis(series)
    }

def aggregate_moments(data):
    moments_data = {'mean': [], 'variance': [], 'skewness': [], 'kurtosis': []}
    
    # Calculate moments for each time series
    for series in data:
        moments = calculate_moments(series)
        for key in moments:
            moments_data[key].append(moments[key])
    
    # Calculate averages and standard deviations for each moment
    aggregated_moments = {}
    for key in moments_data:
        aggregated_moments[f'avg_{key}'] = np.mean(moments_data[key])
        aggregated_moments[f'std_{key}'] = np.std(moments_data[key])
    
    return aggregated_moments

# Calculate aggregated moments for both series
aggregated_moments_pump = aggregate_moments(pump_series)
aggregated_moments_random = aggregate_moments(random_series)

# Print aggregated moments
print("Aggregated Moments for Pump Series:")
for key, value in aggregated_moments_pump.items():
    print(f"{key}: {value:.4f}")

print("\nAggregated Moments for Random Series:")
for key, value in aggregated_moments_random.items():
    print(f"{key}: {value:.4f}")


# Real vs GAN vs VAE vs GP vs Diffusion generated

## Real vs GAN

In [ ]:
print("Real")
print(type(pump_series))
print(pump_series.shape)

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


class Generator(nn.Module):
    def __init__(self, input_size, hidden_dim, output_size):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, hidden_dim),
            nn.ReLU(True),
            nn.Linear(hidden_dim, output_size),
            nn.Tanh()
        )
        
    def forward(self, x):
        return self.fc(x)


def generate_sample(generator, input_size):

    generator.eval()
    with torch.no_grad():
        noise = torch.randn(1, input_size).to(device)
        generated_sample = generator(noise)
        
        plt.figure(figsize=(10, 2))
        plt.plot(generated_sample.cpu().numpy().reshape(-1))
        plt.title("Generated Sample")
        plt.show()


input_size = 100  
hidden_dim = 256
output_size = 1000 

G = Generator(input_size, hidden_dim, output_size).to(device)
G.load_state_dict(torch.load('C:/Users/mahrouaa/CS283_Project/DataPump/CS283/pumpGAN.pth'))

G.eval()

generate_sample(G, input_size)



In [ ]:
import pandas as pd
import numpy as np

def generate_multiple_samples(generator, input_size, num_samples):
    generator.eval()
    with torch.no_grad():
        noise = torch.randn(num_samples, input_size).to(device)
        generated_samples = generator(noise)
        return generated_samples.cpu().numpy().T  # Transpose to make each series a column

num_samples = 466
generated_data = generate_multiple_samples(G, input_size, num_samples)

df_generated = pd.DataFrame(generated_data)
df_generated.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df_generated

In [ ]:
gan_series = df_generated.to_numpy().T
print("GAN")
print(type(gan_series))
print(gan_series.shape)

In [ ]:
import numpy as np
import pandas as pd
import os
from scipy.spatial.distance import cityblock, cosine
from numpy.linalg import norm
from scipy.stats import pearsonr, wasserstein_distance, ks_2samp, moment, skew, kurtosis
from tqdm import tqdm

def calculate_metrics(series1, series2):
    euclidean_distance = norm(series1 - series2)
    manhattan_distance = cityblock(series1, series2)
    cosine_sim = 1 - cosine(series1, series2)
    correlation, _ = pearsonr(series1, series2)
    emd = wasserstein_distance(series1, series2)
    ks_stat, ks_pvalue = ks_2samp(series1, series2)
    return euclidean_distance, manhattan_distance, cosine_sim, correlation, emd, ks_stat, ks_pvalue

def calculate_group_metrics(group1, group2):
    distances = []
    for series1 in tqdm(group1):
        for series2 in group2:
            metrics = calculate_metrics(series1, series2)
            distances.append(metrics)
    distances = np.array(distances)
    means = np.mean(distances, axis=0)
    stds = np.std(distances, axis=0)
    return means, stds

def print_metrics(title, means, stds):
    metrics_names = ["Euclidean", "Manhattan", "Cosine Similarity", "Pearson Correlation", "Earth Mover's", "KS Statistic"]
    print(title + " Averages and Standard Deviations:")
    for i, name in enumerate(metrics_names):
        print(f"{name}:")
        print("  Mean:", means[i], "  Standard Deviation:", stds[i])

# Calculate and print metrics
means_cross, stds_cross = calculate_group_metrics(pump_series, gan_series)

print_metrics("Cross-Group", means_cross, stds_cross)


In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis

def calculate_moments(series):
    return {
        'mean': np.mean(series),
        'variance': np.var(series),
        'skewness': skew(series),
        'kurtosis': kurtosis(series)
    }

def aggregate_moments(data):
    moments_data = {'mean': [], 'variance': [], 'skewness': [], 'kurtosis': []}
    
    # Calculate moments for each time series
    for series in data:
        moments = calculate_moments(series)
        for key in moments:
            moments_data[key].append(moments[key])
    
    # Calculate averages and standard deviations for each moment
    aggregated_moments = {}
    for key in moments_data:
        aggregated_moments[f'avg_{key}'] = np.mean(moments_data[key])
        aggregated_moments[f'std_{key}'] = np.std(moments_data[key])
    
    return aggregated_moments

# Calculate aggregated moments for both series
aggregated_moments_pump = aggregate_moments(pump_series)
aggregated_moments_random = aggregate_moments(gan_series)

# Print aggregated moments
print("Aggregated Moments for Pump Series:")
for key, value in aggregated_moments_pump.items():
    print(f"{key}: {value:.4f}")

print("\nAggregated Moments for Random Series:")
for key, value in aggregated_moments_random.items():
    print(f"{key}: {value:.4f}")


## Real vs VAE

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

class VAE(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size, dropout_rate=0.5):
        super(VAE, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, latent_size * 2)  
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, input_size)
        )
        
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x):
        x = self.encoder(x)
        mu, logvar = torch.chunk(x, 2, dim=-1)
        
        z = self.reparameterize(mu, logvar)
        
        x_recon = self.decoder(z)
        return x_recon, mu, logvar

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
latent_size=64

vae = VAE(input_size=1000, hidden_size=256, latent_size=64).to(device)
vae.load_state_dict(torch.load('C:/Users/mahrouaa/CS283_Project/DataPump/CS283/pumpVAE.pth'))
vae.eval()

def generate_sample(vae, latent_size):
    with torch.no_grad():
        z = torch.randn(1, latent_size).to(device)
        generated_time_series = vae.decoder(z)
        generated_time_series_cpu = generated_time_series.cpu()
        generated_time_series_np = generated_time_series_cpu.squeeze().numpy()

        plt.figure(figsize=(10, 2))
        plt.plot(generated_time_series_np)
        plt.title("Generated Time Series")
        plt.show()

# Generate a synthetic time series sample
generate_sample(vae, latent_size)


In [ ]:
import pandas as pd
import numpy as np

def generate_multiple_samples(vae, latent_size, num_samples):
    with torch.no_grad():
        z = torch.randn(num_samples, latent_size).to(device)
        generated_time_series = vae.decoder(z)
        generated_time_series_cpu = generated_time_series.cpu()
        return generated_time_series_cpu.numpy().T  # Transpose to make each series a column

# Generate 466 synthetic time series
num_samples = 466
generated_data = generate_multiple_samples(vae, latent_size, num_samples)

# Create a DataFrame
df_generated = pd.DataFrame(generated_data)
df_generated.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df_generated

In [ ]:
vae_series = df_generated.to_numpy().T
print("VAE")
print(type(vae_series))
print(vae_series.shape)

In [ ]:
import numpy as np
import pandas as pd
import os
from scipy.spatial.distance import cityblock, cosine
from numpy.linalg import norm
from scipy.stats import pearsonr, wasserstein_distance, ks_2samp, moment, skew, kurtosis
from tqdm import tqdm

def calculate_metrics(series1, series2):
    euclidean_distance = norm(series1 - series2)
    manhattan_distance = cityblock(series1, series2)
    cosine_sim = 1 - cosine(series1, series2)
    correlation, _ = pearsonr(series1, series2)
    emd = wasserstein_distance(series1, series2)
    ks_stat, ks_pvalue = ks_2samp(series1, series2)
    return euclidean_distance, manhattan_distance, cosine_sim, correlation, emd, ks_stat, ks_pvalue

def calculate_group_metrics(group1, group2):
    distances = []
    for series1 in tqdm(group1):
        for series2 in group2:
            metrics = calculate_metrics(series1, series2)
            distances.append(metrics)
    distances = np.array(distances)
    means = np.mean(distances, axis=0)
    stds = np.std(distances, axis=0)
    return means, stds

def print_metrics(title, means, stds):
    metrics_names = ["Euclidean", "Manhattan", "Cosine Similarity", "Pearson Correlation", "Earth Mover's", "KS Statistic"]
    print(title + " Averages and Standard Deviations:")
    for i, name in enumerate(metrics_names):
        print(f"{name}:")
        print("  Mean:", means[i], "  Standard Deviation:", stds[i])

# Calculate and print metrics
means_cross, stds_cross = calculate_group_metrics(pump_series, vae_series)

print_metrics("Cross-Group", means_cross, stds_cross)


In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis

def calculate_moments(series):
    return {
        'mean': np.mean(series),
        'variance': np.var(series),
        'skewness': skew(series),
        'kurtosis': kurtosis(series)
    }

def aggregate_moments(data):
    moments_data = {'mean': [], 'variance': [], 'skewness': [], 'kurtosis': []}
    
    # Calculate moments for each time series
    for series in data:
        moments = calculate_moments(series)
        for key in moments:
            moments_data[key].append(moments[key])
    
    # Calculate averages and standard deviations for each moment
    aggregated_moments = {}
    for key in moments_data:
        aggregated_moments[f'avg_{key}'] = np.mean(moments_data[key])
        aggregated_moments[f'std_{key}'] = np.std(moments_data[key])
    
    return aggregated_moments

# Calculate aggregated moments for both series
aggregated_moments_pump = aggregate_moments(pump_series)
aggregated_moments_random = aggregate_moments(vae_series)

# Print aggregated moments
print("Aggregated Moments for Pump Series:")
for key, value in aggregated_moments_pump.items():
    print(f"{key}: {value:.4f}")

print("\nAggregated Moments for Random Series:")
for key, value in aggregated_moments_random.items():
    print(f"{key}: {value:.4f}")


## Real vs GP

In [ ]:
import os
import pandas as pd
import numpy as np

folder_path = r"C:\Users\mahrouaa\CS283_Project\DataPump\CS283\new_1m_binance"

returns_list = []

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path, usecols=['returns'])
        returns_list.append(df)

all_returns_df = pd.concat(returns_list, axis=1)
all_returns_df = all_returns_df.drop(all_returns_df.index[0])
all_returns_df

In [ ]:
mean_vector = all_returns_df.mean(axis=1)
stdev_vector = all_returns_df.std(axis=1)

df_generated = pd.DataFrame()

initial_price=100
num_series=466

for series_idx in range(num_series):
    return_vector = np.random.normal(mean_vector, stdev_vector)
    price_vector = [initial_price]
    
    for ret in return_vector:
        new_price = price_vector[-1] * (1 + (ret / 100))
        price_vector.append(new_price)
    
    df_generated[f'Series_{series_idx+1}'] = price_vector

df_generated.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df_generated

In [ ]:
gp_series = df_generated.to_numpy().T
print("GP")
print(type(gp_series))
print(gp_series.shape)

In [ ]:
import numpy as np
import pandas as pd
import os
from scipy.spatial.distance import cityblock, cosine
from numpy.linalg import norm
from scipy.stats import pearsonr, wasserstein_distance, ks_2samp, moment, skew, kurtosis
from tqdm import tqdm

def calculate_metrics(series1, series2):
    euclidean_distance = norm(series1 - series2)
    manhattan_distance = cityblock(series1, series2)
    cosine_sim = 1 - cosine(series1, series2)
    correlation, _ = pearsonr(series1, series2)
    emd = wasserstein_distance(series1, series2)
    ks_stat, ks_pvalue = ks_2samp(series1, series2)
    return euclidean_distance, manhattan_distance, cosine_sim, correlation, emd, ks_stat, ks_pvalue

def calculate_group_metrics(group1, group2):
    distances = []
    for series1 in tqdm(group1):
        for series2 in group2:
            metrics = calculate_metrics(series1, series2)
            distances.append(metrics)
    distances = np.array(distances)
    means = np.mean(distances, axis=0)
    stds = np.std(distances, axis=0)
    return means, stds

def print_metrics(title, means, stds):
    metrics_names = ["Euclidean", "Manhattan", "Cosine Similarity", "Pearson Correlation", "Earth Mover's", "KS Statistic"]
    print(title + " Averages and Standard Deviations:")
    for i, name in enumerate(metrics_names):
        print(f"{name}:")
        print("  Mean:", means[i], "  Standard Deviation:", stds[i])

# Calculate and print metrics
means_cross, stds_cross = calculate_group_metrics(pump_series, gp_series)

print_metrics("Cross-Group", means_cross, stds_cross)


In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis

def calculate_moments(series):
    return {
        'mean': np.mean(series),
        'variance': np.var(series),
        'skewness': skew(series),
        'kurtosis': kurtosis(series)
    }

def aggregate_moments(data):
    moments_data = {'mean': [], 'variance': [], 'skewness': [], 'kurtosis': []}
    
    # Calculate moments for each time series
    for series in data:
        moments = calculate_moments(series)
        for key in moments:
            moments_data[key].append(moments[key])
    
    # Calculate averages and standard deviations for each moment
    aggregated_moments = {}
    for key in moments_data:
        aggregated_moments[f'avg_{key}'] = np.mean(moments_data[key])
        aggregated_moments[f'std_{key}'] = np.std(moments_data[key])
    
    return aggregated_moments

# Calculate aggregated moments for both series
aggregated_moments_pump = aggregate_moments(pump_series)
aggregated_moments_random = aggregate_moments(gp_series)

# Print aggregated moments
print("Aggregated Moments for Pump Series:")
for key, value in aggregated_moments_pump.items():
    print(f"{key}: {value:.4f}")

print("\nAggregated Moments for Random Series:")
for key, value in aggregated_moments_random.items():
    print(f"{key}: {value:.4f}")


## Real vs Diffusion

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

file_path = "C:\\Users\\mahrouaa\\Downloads\\generated_time_series.csv"
data = pd.read_csv(file_path)

sampled_data = data.sample(466, random_state=42)

df_generated = sampled_data
new_column_names = [f"Gen_{i+1}" for i in range(df_generated.shape[1])]
df_generated.columns = new_column_names

scaler = MinMaxScaler()
df_generated = pd.DataFrame(scaler.fit_transform(df_generated.T))

df_generated.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df_generated


In [ ]:
diff_series = df_generated.to_numpy().T
print("Diffusion")
print(type(diff_series))
print(diff_series.shape)

In [ ]:
import numpy as np
import pandas as pd
import os
from scipy.spatial.distance import cityblock, cosine
from numpy.linalg import norm
from scipy.stats import pearsonr, wasserstein_distance, ks_2samp, moment, skew, kurtosis
from tqdm import tqdm

def calculate_metrics(series1, series2):
    euclidean_distance = norm(series1 - series2)
    manhattan_distance = cityblock(series1, series2)
    cosine_sim = 1 - cosine(series1, series2)
    correlation, _ = pearsonr(series1, series2)
    emd = wasserstein_distance(series1, series2)
    ks_stat, ks_pvalue = ks_2samp(series1, series2)
    return euclidean_distance, manhattan_distance, cosine_sim, correlation, emd, ks_stat, ks_pvalue

def calculate_group_metrics(group1, group2):
    distances = []
    for series1 in tqdm(group1):
        for series2 in group2:
            metrics = calculate_metrics(series1, series2)
            distances.append(metrics)
    distances = np.array(distances)
    means = np.mean(distances, axis=0)
    stds = np.std(distances, axis=0)
    return means, stds

def print_metrics(title, means, stds):
    metrics_names = ["Euclidean", "Manhattan", "Cosine Similarity", "Pearson Correlation", "Earth Mover's", "KS Statistic"]
    print(title + " Averages and Standard Deviations:")
    for i, name in enumerate(metrics_names):
        print(f"{name}:")
        print("  Mean:", means[i], "  Standard Deviation:", stds[i])

# Calculate and print metrics
means_cross, stds_cross = calculate_group_metrics(pump_series, diff_series)

print_metrics("Cross-Group", means_cross, stds_cross)


In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis

def calculate_moments(series):
    return {
        'mean': np.mean(series),
        'variance': np.var(series),
        'skewness': skew(series),
        'kurtosis': kurtosis(series)
    }

def aggregate_moments(data):
    moments_data = {'mean': [], 'variance': [], 'skewness': [], 'kurtosis': []}
    
    # Calculate moments for each time series
    for series in data:
        moments = calculate_moments(series)
        for key in moments:
            moments_data[key].append(moments[key])
    
    # Calculate averages and standard deviations for each moment
    aggregated_moments = {}
    for key in moments_data:
        aggregated_moments[f'avg_{key}'] = np.mean(moments_data[key])
        aggregated_moments[f'std_{key}'] = np.std(moments_data[key])
    
    return aggregated_moments

# Calculate aggregated moments for both series
aggregated_moments_pump = aggregate_moments(pump_series)
aggregated_moments_random = aggregate_moments(diff_series)

# Print aggregated moments
print("Aggregated Moments for Pump Series:")
for key, value in aggregated_moments_pump.items():
    print(f"{key}: {value:.4f}")

print("\nAggregated Moments for Random Series:")
for key, value in aggregated_moments_random.items():
    print(f"{key}: {value:.4f}")
